<a href="https://colab.research.google.com/github/egoist-suiluj/Ventify/blob/main/ventify_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask flask-ngrok google-genai firebase-admin

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from google import genai
import firebase_admin
from firebase_admin import credentials, firestore
import os
from google.colab import files
from google.colab import userdata

print("Installations at Imports ay kompleto na. Ready na para sa key conenctions")

Installations at Imports ay kompleto na. Ready na para sa key conenctions


In [2]:
#First step - Upload Firebase key

print("Upload firebase key")
db=None
try:
  #mag bubukas ang upload files
  uploaded = files.upload()
  if uploaded:
    key_filename = list(uploaded.keys())[0]
    os.environ['FIREBASE_KEY_PATH'] = key_filename

    #Kinokonekta yung pyhton sa firebase
    if not firebase_admin_apps:
      cred = credentials.Certificate(os.environ.get(FIREBASE_KEY_PATH))
      firebase_admin.initialize_app(cred)
    db = firestore.client()
    print("Firebase (Firestore) Client Initialized.")
  else:
    print("No file uploaded, Hindi kokonekta sa Firebase.")
except Exception as e:
  print(f"Error initalizing Firebase: {e}")


#Step two - E-set up si ang AI engine

client = None
try:
  #Kukuhanin yung key galing sa Colab Secret Panel
  os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
  #Gagamitin ang key para mag-activate ng Gemini Client
  client = genai.Client()
  AI_MODEL = 'gemini-2.5-flash'
  print("Gemini Client Initialized.")
except Exception as e:
  print(f"Error initalizing Gemini Client. Pa-check ng GEMINI_API_KEY {e}")
  client = None


Upload firebase key


Saving ventify-project-firebase-adminsdk-fbsvc-9798eae7e7.json to ventify-project-firebase-adminsdk-fbsvc-9798eae7e7.json
Error initalizing Firebase: name 'firebase_admin_apps' is not defined
Error initalizing Gemini Client. Pa-check ng GEMINI_API_KEY Secret GEMINI_API_KEY does not exist.
